# I. Preface

As a practitioner and enthusiast in simulation, I have always been fascinated by modeling business, economics, politics, and more through programs. In this notebook, I will explore the potential outcomes of integrating Mechanism Design theories with Generative AI by selectively revealing information to agents and observing their reactions.

Through repeated experiments, this approach achieves three key goals:

- Gaining a fresh perspective on GenAI models.
- Understanding how AI agents respond to economic theory-driven rules.
- Most importantly—finding enjoyment in the process (and by 'you,' I really mean 'I').

This is the first step in my exploration, and I’m making it public to invite discussion. If you share a similar interest, feel free to comment and exchange ideas.
In this notebook, we will simulate a bilateral trade with two roles: buyer and seller. Each participant has a private valuation of the good. They must decide on their declared valuation. If the buyer's declared valuation is greater than the seller's, a trade occurs; otherwise, no deal is made.
The game follows the VCG (Vickrey-Clarke-Groves) mechanism, which determines the corresponding calculations, such as the transfer payments for the buyer and seller.

>Mechanism design (sometimes implementation theory or institution design)[1] is a branch of economics and game theory. It studies how to construct rules—called mechanisms or institutions—that produce good outcomes according to some predefined metric, even when the designer does not know the players' true preferences or what information they have. Mechanism design thus focuses on the study of solution concepts for a class of private-information games. 

>For more information about Mechanism Design: https://en.wikipedia.org/wiki/Mechanism_design

>For more information about VCG mechanism: https://en.wikipedia.org/wiki/Vickrey%E2%80%93Clarke%E2%80%93Groves_mechanism

# II. Config rules and information to be revealed to agents

In [1]:
delimiter = "####"
system_message = f"""
This is a bilateral trade game following VCG rules defined in VCG mechanism. \
In this game there are only two roles, buyer and seller. \
buyer and seller have their own evaluation for a good and that is their private information, \
in the meantime they also have declared evaluation. Both buyer and seller want to maximize their own utility. \
Remember, this is simotaneous game, buyer and seller give the declared evaluation at the same time and whether the trade would or wouldn't occur will be decided by that

Step 1:{delimiter} Think about what should be the declared evaluation

Step 2:{delimiter}: State your declared evaluation in the form of ##### Declared_Evaluation=Number #####
"""

In [2]:
buyer_private_valuation = 5  # Buyer knows their own valuation
seller_private_valuation = 3  # Seller knows their own cost

In [3]:
buyer_prompt = f"""
You are a buyer, your private evaluation for the good is {buyer_private_valuation}""".strip()
seller_prompt = f"""
You are a seller, your private evaluation for the good is {seller_private_valuation}""".strip()

In [4]:
buyer_model = 'gemini-2.0-flash-thinking-exp'
seller_model = 'gemini-2.0-flash-thinking-exp'

# III. Technical configuration

In [5]:
from kaggle_secrets import UserSecretsClient
from google import genai


user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Gemini")

client = genai.Client(
    api_key=secret_value_0,
    http_options={'api_version': 'v1alpha'},
)

In [6]:


messages = [
    f"{system_message}",
    f"{buyer_prompt}",
]

response = client.models.generate_content(
    model=buyer_model,
    contents=messages,
)

# print(response.text)

buyer_thoughts = response.text

In [7]:


messages = [
    f"{system_message}",
    f"{seller_prompt}",
]

response = client.models.generate_content(
    model=seller_model,
    contents=messages,
)

# print(response.text)

seller_thoughts = response.text

In [8]:
import re

# Regular expression to match 'Declared_Evaluation = ' followed by a number
match = re.search(r"Declared_Evaluation\s*=\s*(\d+)", buyer_thoughts)

if match:
    buyer_declared = int(match.group(1))  # Extract the number and convert it to an integer
    print(f"Extracted number: {buyer_declared}")
else:
    print("No match found")

# Regular expression to match 'Declared_Evaluation = ' followed by a number
match = re.search(r"Declared_Evaluation\s*=\s*(\d+)", seller_thoughts)

if match:
    seller_declared = int(match.group(1))  # Extract the number and convert it to an integer
    print(f"Extracted number: {seller_declared}")
else:
    print("No match found")

Extracted number: 5
Extracted number: 3


# IV. Trade Outcome and Agents' Reasoning Steps

In [9]:
def vcg_bilateral_trade(buyer_private_valuation, seller_private_valuation,declared_valuation,declared_cost, d=1):
    """Simulates bilateral trade with the VCG mechanism.

    Args:
        buyer_private_valuation: The buyer's true valuation.
        seller_private_valuation: The seller's true cost.
        d: Externality scaling factor.

    Returns:
        Tuple: (trade_occurs, price, buyer_utility, seller_utility)
    """
    # Buyer and seller use LLMs to decide their declared valuations
    buyer_declared_valuation = declared_valuation
    seller_declared_valuation = declared_cost

    # Seller's declared valuation with externality
    seller_declared_true_valuation = -d * seller_declared_valuation

    # Trade decision based on declared valuations
    if buyer_declared_valuation >= -seller_declared_true_valuation:
        trade_occurs = True
    else:
        trade_occurs = False

    if trade_occurs:
        # VCG price based on declared valuations
        price = -seller_declared_true_valuation

        # Buyer's utility (using private valuation)
        buyer_utility = buyer_private_valuation - price

        # Seller's transfer (using declared valuations)
        seller_transfer = d * buyer_declared_valuation - buyer_declared_valuation

        # Seller's utility (using private valuation)
        seller_utility = seller_transfer + (-d * seller_private_valuation)
    else:
        price = 0
        buyer_utility = 0
        seller_utility = -d * seller_private_valuation

    return (trade_occurs, price, buyer_utility, seller_utility, 
            buyer_declared_valuation, seller_declared_valuation)



results = vcg_bilateral_trade(buyer_private_valuation, seller_private_valuation, buyer_declared, seller_declared)

trade_occurs, price, buyer_utility, seller_utility, \
buyer_declared_valuation, seller_declared_valuation = results

print(f"Buyer private valuation: {buyer_private_valuation} \n")
print(f"Buyer declared valuation: {buyer_declared_valuation} \n")
print(f"Seller private valuation (inherent cost): {seller_private_valuation} \n")
print(f"Seller declared valuation (inherent cost): {seller_declared_valuation} \n")
print(f"Buyer thoughts: \n {buyer_thoughts} \n")

Buyer private valuation: 5 

Buyer declared valuation: 5 

Seller private valuation (inherent cost): 3 

Seller declared valuation (inherent cost): 3 

Buyer thoughts: 
 #### Think about what should be the declared evaluation
In a VCG mechanism for bilateral trade, the buyer's goal is to maximize their utility. The trade occurs if the buyer's declared valuation (d_b) is greater than or equal to the seller's declared valuation (d_s). If the trade occurs, the buyer pays the seller's declared valuation (d_s). The buyer's utility is their private valuation (v_b) minus the payment (d_s) if trade occurs, and 0 if trade does not occur.  In this case, my private valuation v_b = 5.

Let's consider the scenarios for different declared evaluations.

If I declare d_b = 5 (truthfully):
- If the seller declares d_s <= 5, trade happens, and my utility is 5 - d_s. To maximize my utility, I want d_s to be as small as possible while still ensuring trade happens. However, I cannot control d_s.
- If the s

In [10]:
print(f"Seller thoughts: \n {seller_thoughts} \n")

Seller thoughts: 
 Step 1:#### Think about what should be the declared evaluation
In a VCG mechanism for bilateral trade, truthful revelation is a weakly dominant strategy. For a seller, if they declare their true private evaluation, their utility is 0 when a trade occurs (declared price - private evaluation = 0) and 0 when no trade occurs.

Let's consider the cases for declaring a value different from the private evaluation:

Case 1: Declare a value higher than the private evaluation (d_s > 3).
If the buyer's declared evaluation (d_b) is greater than or equal to this higher declared value (d_b >= d_s > 3), trade occurs. The seller's utility will be d_s - 3 > 0 (positive utility).
If the buyer's declared evaluation (d_b) is less than this higher declared value (d_b < d_s), no trade occurs. The seller's utility will be 0.
By declaring a higher value, the seller risks no trade occurring if the buyer's valuation is not high enough, but gains positive utility if trade happens.

Case 2: Dec

In [11]:
print(f"Trade occurs: {trade_occurs} \n")
if trade_occurs:
    print(f"Price: {price}")
    print(f"Buyer utility: {buyer_utility}")
    print(f"Seller utility: {seller_utility}")
else:
    print(f"Price: {price}")
    print(f"Buyer utility: {buyer_utility}")
    print(f"Seller utility: {seller_utility}")

Trade occurs: True 

Price: 3
Buyer utility: 2
Seller utility: -3


# X. References and useful information

https://ai.google.dev/gemini-api/docs/thinking

pip install --upgrade google-generativeai

?client.models.generate_content